In [1]:
import uvage
from random import randrange
import os.path

#GLobal Variables ----------------------------------------------------------------
score = 0
game_on = False
game_over = False
game_over_screen = False
player_name = 'z'
scoreboard_values = []

#Set-up Game Objects ----------------------------------------------------------------
camera = uvage.Camera(600, 600)
scoreboard = uvage.from_color(300, 300, 'white', 400, 400)
button = uvage.from_color(300, 500, 'red', 400, 20)
score_table = []
current_frame = 0

coin = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), r'Snake Sprite Sheets\coin.png')

apple_image = uvage.load_sprite_sheet(r'Snake Sprite Sheets\apple_float.png', 3, 3)
apple = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), apple_image[0])
apple.size = [50, 50]

snake_headsheet = uvage.load_sprite_sheet(r'Snake Sprite Sheets\snake eat.png', 2, 2)

snake_head = uvage.from_image(300, 300, snake_headsheet[0])
snake_head.size = [100, 100]
snake_head.speed = [5, 0]

facing_right = True
facing_up = True

snake_body = uvage.load_sprite_sheet(r'Snake Sprite Sheets\snake_body.png', 1, 1) #loads the sprite sheet

previous_snakelocation = (snake_head.x, snake_head.y)

snake_list = []
snake_size = 5

for body in range(snake_size):
    snake = uvage.from_image(previous_snakelocation[0] - 20, 300, snake_body[0])
    snake.size = [50, 50]
    previous_snakelocation = (snake.x, snake.y)
    snake_list.append(snake)



#Inter-Session Progress ----------------------------------------------------------------

def store_highscore():

    global player_name, score

    if os.path.isfile('highscore.txt') == False:
        file = open('highscore.txt', 'x')
        file.write(player_name + ',' + str(score))
        file.close()

    highscore_file = open('highscore.txt', 'a')
    highscore_file.write('\n' + player_name + ',' + str(score))
    highscore_file.close()

    return

def clear_highscore():
    highscore_file = open('highscore.txt', 'w')
    print(file = highscore_file)
    highscore_file.close()

    return

def retrieve_highscorelist():

    highscore_file = open('highscore.txt', 'r')
    highscore = []
    
    for row in highscore_file.read().split('\n'):
        highscore.append(row.split(','))

    highscore_file.close()

    highscore.sort(key = lambda x: int(x[1]), reverse = True)
    
    return highscore 

def create_score_table(values):
    global score_table
    
    scoreboard_values = values
    score_table = []

    rank = 1
    entries = 0 

    while entries < 10 and entries <= (len(scoreboard_values) - 2):
        text = "{} | Score:{}".format(scoreboard_values[rank][0], scoreboard_values[rank][1])
        entry_diplay = uvage.from_text(scoreboard.x, scoreboard.top - 20 + 40 * rank, text, 40, "black")
        score_table.append(entry_diplay)
        rank += 1
        entries += 1

    return

# Collectibles -------------------------------

def handle_coin():
    global score, coin

    camera.draw(coin)

    if coin.touches(snake_head):
        coin = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), r'Snake Sprite Sheets\coin.png')
        score += 100

    return


def handle_apple():
    global score, apple, apple_image, current_frame, snake_list, snake_body

    apple = uvage.from_image(apple.x, apple.y, apple_image[current_frame%9])
    apple.size = [50, 50]
    camera.draw(apple)

    if apple.touches(snake_head):
        apple = uvage.from_image(randrange(50, 550, 50), randrange(50, 550, 50), apple_image[0])
        apple.size = [50, 50]
        score += 100
        snake_body = uvage.from_image(snake_head.x, snake_head.y, r'Snake Sprite Sheets\snake_body.png')
        snake_body.size = [50,50]
        snake_list.append(snake_body)


    return

def score_display():
	global score

	camera.draw("Score: " + str(int(score)), 36, "white", 300, 50) #displays the score while they're playing the game

# Snake --------------------------------

def handle_snake():
    global facing_right, facing_up, game_over

    if game_over == True:
        snake_head.xspeed = 0
        snake_head.yspeed = 0

    if game_on == True:

        if snake_head.x < 0 or snake_head.x > 800 or snake_head.y < 0 or snake_head.y > 800:
            game_over = True
            store_highscore()

        if uvage.is_pressing("right arrow"):
            if not facing_right:        #if not facing right it flips it so that it is
                facing_right = True
                snake_head.flip()
            snake_head.xspeed = 5
            snake_head.yspeed = 0
                
        elif uvage.is_pressing("left arrow"):
            if facing_right:            #flips snake body to be facing in the left direction
                facing_right = False
                snake_head.flip()
            snake_head.xspeed = -5
            snake_head.yspeed = 0

        elif uvage.is_pressing("up arrow"):
            if not facing_up:   #IMPORTANT: Still needs work, can't figure out how to flip/rotate up and down---------
                facing_up = True
                snake_head.rotate(180)
                snake_head.flip()
            snake_head.xspeed = 0
            snake_head.yspeed = -5
            snake_head.rotate(90)

        elif uvage.is_pressing("down arrow"):
            if not facing_up:   #IMPORTANT: Still needs work, can't figure out how to flip/rotate up and down---------
                facing_up = True
                snake_head.rotate(180)
                snake_head.flip()
            snake_head.xspeed = 0
            snake_head.yspeed = 5      
        
        camera.draw(snake_head)
        snake_head.move_speed()

        for body in snake_list:
            camera.draw(body)
            body.move_speed()

    
    return


# Restart -----------------------------------------
def handle_gameover():
    global game_over, game_on, game_over_screen, scoreboard_values

    scoreboard_values = retrieve_highscorelist()
    create_score_table(scoreboard_values)
    game_on = False
    
    return

def handle_gameover_screen():
    global game_over, game_on, game_over_screen, score_table

    camera.draw(scoreboard)
    camera.draw(button)
    camera.draw("Close", 36, "white", 300, 500)
    for row in score_table:
        camera.draw(row)
    if button.contains(camera.mouse) and camera.mouseclick == True:
        camera.clear('light blue')
        restart_game()
        game_over = False
    

    return

def restart_game():
    global facing_right, facing_up, score, snake_list, snake_head

    score = 0

    snake_head.x = 300
    snake_head.y = 300

    snake_head.size = [100, 100]
    snake_head.speed = [5, 0]

    facing_right = True
    facing_up = True

    snake_body = uvage.load_sprite_sheet(r'Snake Sprite Sheets\snake_body.png', 1, 1) #loads the sprite sheet

    previous_snakelocation = (snake_head.x, snake_head.y)

    snake_list = []
    snake_size = 5

    for body in range(snake_size):
        snake = uvage.from_image(previous_snakelocation[0] - 20, 300, snake_body[0])
        snake.size = [50, 50]
        previous_snakelocation = (snake.x, snake.y)
        snake_list.append(snake)
    return

# Game Code ----------------------------------------------------------------

def game():
    global snake, current_frame, game_on

    camera.clear('light blue')

    if game_on == False:
        camera.draw("Hit Space to Start!", 100, "white", 300, 300)

    if uvage.is_pressing('space'):
        game_on = True

    if game_on == True:
        score_display()
        handle_snake()
        handle_apple()
        handle_coin()
        current_frame += 1

    if game_over == True:
        handle_gameover()
        handle_gameover_screen()

    camera.display()
    return

#Run Game ----------------------------------------------------------------
uvage.timer_loop(30, game)

pygame 2.1.2 (SDL 2.0.18, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
